In [1]:
import sys
import h2o
import pandas as pd
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /home/borja/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp2klcp_kp
  JVM stdout: /tmp/tmp2klcp_kp/h2o_borja_started_from_python.out
  JVM stderr: /tmp/tmp2klcp_kp/h2o_borja_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.31.0.5213
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_borja_wekhcw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.881 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
df = h2o.import_file(path="output/train_all.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
df.shape

(40455, 14)

In [5]:
display(df.head())

id,carat,cut,color,clarity,depth,table,x,y,z,price,color2,cut2,clarity2
0,1.12,Premium,H,SI1,61.6,59,6.67,6.63,4.1,5363,4,3,2
1,1.14,Very Good,E,SI2,60,54,6.74,6.97,4.11,5593,1,4,3
2,0.9,Very Good,D,SI2,60.3,63,6.12,6.22,3.72,3534,0,4,3
3,0.71,Ideal,E,VS1,61.9,54,5.74,5.76,3.56,3212,1,2,4
4,0.34,Very Good,F,SI2,60,62,4.51,4.55,2.72,447,2,4,3
5,0.33,Very Good,E,VS2,63.1,57,4.38,4.43,2.78,672,1,4,5
6,0.55,Premium,I,VVS1,61.3,59,5.27,5.3,3.24,1593,5,3,6
7,1.19,Premium,D,SI2,58.8,58,7.04,6.9,4.1,5504,0,3,3
8,0.41,Premium,E,VS1,60.4,58,4.78,4.82,2.9,1079,1,3,4
9,0.3,Premium,G,SI2,62.5,59,4.27,4.31,2.68,394,3,3,3


In [6]:
test = h2o.import_file(path="inputs/predict.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
test = test.as_data_frame()

In [8]:

test = test[["carat","cut","color","clarity","x","y","z"]]
test.head()

,carat,cut,color,clarity,x,y,z
0,0.33,Very Good,I,IF,4.44,4.46,2.76
1,1.21,Very Good,D,SI2,6.77,6.83,4.24
2,1.06,Very Good,D,SI1,6.64,6.71,3.96
3,0.36,Ideal,E,VVS1,4.64,4.61,2.54
4,0.70,Ideal,E,VS1,5.67,5.72,3.55


In [9]:
def cutchanger(x):
    if x== "Ideal":
        return 2
    elif x== "Premium":
        return 3
    elif x == "Very Good":
        return 4
    elif x== "Good":
        return 1
    else:
        return 0
    
def colorchanger(x):
    if x== "G":
        return 3
    elif x== "E":
        return 1
    elif x == "F":
        return 2
    elif x== "H":
        return 4
    elif x== "D":
        return 0
    elif x== "I":
        return 5
    else:
        return 6
    
def clarytychanger(x):
    if x== "SI1":
        return 2
    elif x== "VS2":
        return 5
    elif x == "SI2":
        return 3
    elif x== "VS1":
        return 4
    elif x== "VVS2":
        return 7
    elif x== "VVS1":
        return 6
    elif x== "IF":
        return 1
    else:
        return 0

In [10]:
test["cut2"] = test["cut"].apply(cutchanger)
test["color2"] = test["color"].apply(colorchanger)
test["clarity2"] = test["clarity"].apply(clarytychanger)

In [11]:
test.head()

,carat,cut,color,clarity,x,y,z,cut2,color2,clarity2
0,0.33,Very Good,I,IF,4.44,4.46,2.76,4,5,1
1,1.21,Very Good,D,SI2,6.77,6.83,4.24,4,0,3
2,1.06,Very Good,D,SI1,6.64,6.71,3.96,4,0,2
3,0.36,Ideal,E,VVS1,4.64,4.61,2.54,2,1,6
4,0.70,Ideal,E,VS1,5.67,5.72,3.55,2,1,4


In [12]:
response= "price"

In [13]:
predictors = ["carat","color2","cut2","clarity2","x","y","z"]

In [14]:
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.automl import H2OAutoML
diamons_model2 = H2OAutoML(verbosity = "warn",max_runtime_secs=1000)
#for i in range(0,5):
train, valid = df.split_frame(ratios = [.8])
diamons_model2.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid
                   )


AutoML progress: |
10:10:56.958: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [15]:
diamons_model2.leaderboard


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_101056,293484,541.741,293484,272.106,0.0944038
StackedEnsemble_BestOfFamily_AutoML_20201006_101056,303168,550.607,303168,283.077,0.102585
GBM_1_AutoML_20201006_101056,309331,556.175,309331,287.447,0.105818
GBM_2_AutoML_20201006_101056,310085,556.853,310085,285.783,nan
GBM_3_AutoML_20201006_101056,310365,557.105,310365,283.905,nan
GBM_4_AutoML_20201006_101056,310992,557.667,310992,281.266,nan
GBM_grid__1_AutoML_20201006_101056_model_3,312842,559.323,312842,293.632,0.110415
GBM_grid__1_AutoML_20201006_101056_model_10,317760,563.702,317760,291.681,0.106114
GBM_grid__1_AutoML_20201006_101056_model_11,318202,564.094,318202,280.048,0.0942683
GBM_grid__1_AutoML_20201006_101056_model_13,320400,566.039,320400,286.553,nan


In [16]:
display(test)
test = h2o.H2OFrame(test[["carat","color2","cut2","clarity2","x","y","z"]].reset_index())
pred = diamons_model2.leader.predict(test)
pred

,carat,cut,color,clarity,x,y,z,cut2,color2,clarity2
0,0.33,Very Good,I,IF,4.44,4.46,2.76,4,5,1
1,1.21,Very Good,D,SI2,6.77,6.83,4.24,4,0,3
2,1.06,Very Good,D,SI1,6.64,6.71,3.96,4,0,2
3,0.36,Ideal,E,VVS1,4.64,4.61,2.54,2,1,6
4,0.70,Ideal,E,VS1,5.67,5.72,3.55,2,1,4
...,...,...,...,...,...,...,...,...,...,...
13480,2.04,Good,D,SI1,8.15,8.11,5.03,1,0,2
13481,0.31,Ideal,G,IF,4.37,4.39,2.70,2,3,1
13482,0.52,Ideal,D,VVS2,5.14,5.18,3.20,2,0,7
13483,0.50,Premium,F,SI1,5.23,5.19,3.08,3,2,2


Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
745.944
6008.97
5895.4
1097.44
3210.55
520.095
1458.32
858.978
8636.47
651.437


In [17]:
test["price"] = pred
test.head()

index,carat,color2,cut2,clarity2,x,y,z,price
0,0.33,5,4,1,4.44,4.46,2.76,745.944
1,1.21,0,4,3,6.77,6.83,4.24,6008.97
2,1.06,0,4,2,6.64,6.71,3.96,5895.4
3,0.36,1,2,6,4.64,4.61,2.54,1097.44
4,0.7,1,2,4,5.67,5.72,3.55,3210.55
5,0.3,5,3,7,4.28,4.37,2.67,520.095
6,0.52,1,3,2,5.3,5.21,3.1,1458.32
7,0.32,3,2,6,4.39,4.4,2.72,858.978
8,1.51,2,1,3,7.22,7.28,4.56,8636.47
9,0.42,6,2,5,4.79,4.86,2.96,651.437


In [18]:
upload = test[["index","price"]]

In [19]:
upload.rename(columns={'index': 'id'})
upload

id,price
0,745.944
1,6008.97
2,5895.4
3,1097.44
4,3210.55
5,520.095
6,1458.32
7,858.978
8,8636.47
9,651.437


In [20]:
upload = upload.as_data_frame()

In [21]:


upload

,id,price
0,0,745.943936
1,1,6008.969826
2,2,5895.399679
3,3,1097.442395
4,4,3210.545945
...,...,...
13480,13480,17116.726537
13481,13481,905.576554
13482,13482,2567.229845
13483,13483,1310.818985


In [22]:
upload.to_csv(r'output/pred4.csv',index=False,header=True)

In [23]:
df2 = h2o.import_file(path="inputs/train.csv")
test2 = h2o.import_file(path="inputs/predict.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
response= "price"

In [25]:
predictors = ["carat","color","cut","clarity","x","y","z"]

In [26]:
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.automl import H2OAutoML
diamons_model3 = H2OAutoML(verbosity = "warn",max_runtime_secs=1000)
#for i in range(0,5):
train, valid = df2.split_frame(ratios = [.8])
diamons_model3.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid
                   )


AutoML progress: |
10:26:56.840: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [27]:
diamons_model3.leaderboard


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_102656,293003,541.297,293003,271.537,0.0947137
StackedEnsemble_BestOfFamily_AutoML_20201006_102656,297287,545.24,297287,274.719,0.097639
GBM_3_AutoML_20201006_102656,299560,547.321,299560,277.227,0.101466
GBM_2_AutoML_20201006_102656,300585,548.256,300585,281.182,0.104264
GBM_1_AutoML_20201006_102656,304010,551.371,304010,283.839,0.103436
GBM_grid__1_AutoML_20201006_102656_model_11,308191,555.149,308191,282.331,nan
GBM_4_AutoML_20201006_102656,310596,557.311,310596,279.7,nan
GBM_grid__1_AutoML_20201006_102656_model_5,311270,557.915,311270,280.582,0.0960986
GBM_grid__1_AutoML_20201006_102656_model_7,311440,558.068,311440,278.217,0.100139
GBM_grid__1_AutoML_20201006_102656_model_1,311453,558.08,311453,291.139,0.110513


In [28]:
predictors2 = ["carat","color","cut","clarity","depth","table","x","y","z"]

In [29]:
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.automl import H2OAutoML
diamons_model4 = H2OAutoML(verbosity = "warn",max_runtime_secs=1000)
#for i in range(0,5):
train, valid = df2.split_frame(ratios = [.8])
diamons_model4.train(x=predictors2,
               y=response,
               training_frame=train,
               validation_frame=valid
                   )


AutoML progress: |
10:42:20.392: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [30]:
diamons_model4.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_104220,290044,538.557,290044,268.706,0.0924219
StackedEnsemble_BestOfFamily_AutoML_20201006_104220,298504,546.355,298504,274.725,0.0979218
GBM_grid__1_AutoML_20201006_104220_model_10,303577,550.978,303577,279.868,0.101177
GBM_3_AutoML_20201006_104220,306367,553.505,306367,279.427,0.101002
GBM_grid__1_AutoML_20201006_104220_model_5,307406,554.442,307406,274.684,0.0910949
GBM_1_AutoML_20201006_104220,308465,555.396,308465,285.097,0.106395
GBM_4_AutoML_20201006_104220,309993,556.77,309993,278.246,0.0968065
GBM_grid__1_AutoML_20201006_104220_model_17,311826,558.414,311826,276.27,0.0907757
GBM_2_AutoML_20201006_104220,313155,559.602,313155,285.599,nan
GBM_grid__1_AutoML_20201006_104220_model_11,317506,563.477,317506,290.612,nan


In [31]:
df3 = h2o.import_file(path="output/train_all.csv")
test3 = h2o.import_file(path="inputs/predict.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [32]:
test3 = test3.as_data_frame()

In [33]:
test3["cut"] = test3["cut"].apply(cutchanger)
test3["color"] = test3["color"].apply(colorchanger)
test3["clarity"] = test3["clarity"].apply(clarytychanger)

In [34]:
response= "price"

In [35]:
predictors = ["carat","color","cut","clarity","table","depth","x","y","z"]

In [36]:
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.automl import H2OAutoML
diamons_model5 = H2OAutoML(verbosity = "warn",max_runtime_secs=1000)
#for i in range(0,5):
train, valid = df.split_frame(ratios = [.8])
diamons_model5.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid
                   )


AutoML progress: |
10:58:17.442: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [37]:
diamons_model5.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20201006_105817,295559,543.654,295559,275.389,0.0987917
StackedEnsemble_AllModels_AutoML_20201006_105817,298895,546.713,298895,270.791,0.0946413
GBM_grid__1_AutoML_20201006_105817_model_18,299190,546.983,299190,279.594,0.101345
GBM_grid__1_AutoML_20201006_105817_model_8,299389,547.164,299389,278.482,0.098662
GBM_3_AutoML_20201006_105817,303743,551.129,303743,277.661,0.0985703
GBM_4_AutoML_20201006_105817,304685,551.983,304685,276.092,0.0994334
GBM_2_AutoML_20201006_105817,304756,552.048,304756,282.409,0.101442
GBM_1_AutoML_20201006_105817,306413,553.546,306413,284.486,0.102797
GBM_grid__1_AutoML_20201006_105817_model_4,310609,557.323,310609,276.768,0.0942858
GBM_grid__1_AutoML_20201006_105817_model_6,310880,557.566,310880,279.558,0.0996992


In [38]:
display(test)
#test = h2o.H2OFrame(test[["carat","color","cut","clarity","x","y","z"]].reset_index())
pred = diamons_model5.leader.predict(test)
pred

index,carat,color2,cut2,clarity2,x,y,z,price
0,0.33,5,4,1,4.44,4.46,2.76,745.944
1,1.21,0,4,3,6.77,6.83,4.24,6008.97
2,1.06,0,4,2,6.64,6.71,3.96,5895.4
3,0.36,1,2,6,4.64,4.61,2.54,1097.44
4,0.7,1,2,4,5.67,5.72,3.55,3210.55
5,0.3,5,3,7,4.28,4.37,2.67,520.095
6,0.52,1,3,2,5.3,5.21,3.1,1458.32
7,0.32,3,2,6,4.39,4.4,2.72,858.978
8,1.51,2,1,3,7.22,7.28,4.56,8636.47
9,0.42,6,2,5,4.79,4.86,2.96,651.437


stackedensemble prediction progress: |████████████████████████████████████| 100%


/home/borja/.local/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'cut': substituting in a column of NaN
  warnings.warn(w)
/home/borja/.local/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'color': substituting in a column of NaN
  warnings.warn(w)
/home/borja/.local/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'clarity': substituting in a column of NaN
  warnings.warn(w)
/home/borja/.local/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'depth': substituting in a column of NaN
  warnings.warn(w)
/home/borja/.local/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'table': substituting in a column of NaN
  warnings.warn(w)


predict
768.316
6851.95
5552.98
853.778
2522.22
665.554
1666.62
734.005
10857.5
1001.6


In [39]:
test["price"] = pred
test.head()

index,carat,color2,cut2,clarity2,x,y,z,price
0,0.33,5,4,1,4.44,4.46,2.76,768.316
1,1.21,0,4,3,6.77,6.83,4.24,6851.95
2,1.06,0,4,2,6.64,6.71,3.96,5552.98
3,0.36,1,2,6,4.64,4.61,2.54,853.778
4,0.7,1,2,4,5.67,5.72,3.55,2522.22
5,0.3,5,3,7,4.28,4.37,2.67,665.554
6,0.52,1,3,2,5.3,5.21,3.1,1666.62
7,0.32,3,2,6,4.39,4.4,2.72,734.005
8,1.51,2,1,3,7.22,7.28,4.56,10857.5
9,0.42,6,2,5,4.79,4.86,2.96,1001.6


In [40]:
upload = test[["index","price"]]

In [41]:
upload.rename(columns={'index': 'id'})
upload

id,price
0,768.316
1,6851.95
2,5552.98
3,853.778
4,2522.22
5,665.554
6,1666.62
7,734.005
8,10857.5
9,1001.6


In [42]:
upload = upload.as_data_frame()

In [43]:
upload

,id,price
0,0,768.315810
1,1,6851.951722
2,2,5552.984559
3,3,853.777755
4,4,2522.222815
...,...,...
13480,13480,14908.611041
13481,13481,723.227519
13482,13482,1667.161744
13483,13483,1589.969207


In [44]:
upload.to_csv(r'output/pred4.csv',index=False,header=True)